In [1]:
##### IMPORTS
%load_ext autoreload
%autoreload 2
import pandas as pd
from ipywidgets import *
import copy
import matplotlib.pyplot as plt
import tsam.timeseriesaggregation as tsam
from IPython.core.display import display, HTML
import base64
from io import StringIO
%matplotlib inline

In [2]:
##### Style
display(HTML("""
        <style>
            .header img {float: right;}
            .header h1 {position: absolute;}
            [data-mime-type='image/png'] {text-align: center;}
        </style>
        """))

In [3]:
##### Header with the logos
display(HTML('<div class="header">\
                  <h1>TSAM GUI</h1>\
                  <img style="width: 10%;" src="https://upload.wikimedia.org/wikipedia/de/8/8b/J%C3%BClich_fz_logo.svg" alt="Juelich Logo"> \
                  <img style="width: 8%;" src="https://www.wiwi.rwth-aachen.de/global/show_picture.asp?id=aaaaaaaaabczicn&w=842&q=77&meta=0" alt="Metis Logo">\
                </div>'))
display(HTML("<hr>"))

### Upload  your data

In [4]:
uploader = widgets.FileUpload(
                accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
                multiple=False,  # True to accept multiple files upload else False
            )
uploader.add_class("right-space-class")
display(HTML("<style>.right-space-class{margin-right: 50px; margin-left: 15px;}</style>"))

In [5]:
button_data = Button(description="Confirm Data", button_style="success", icon='check')

HBox(children=[uploader, button_data])

In [6]:
data_output = Output()
display(data_output)

Output()

### Inputs

In [7]:
steps = widgets.BoundedIntText(
            value=24,
            description='Number of time steps per period',
            disabled=False,
            style={'description_width': 'initial'},
            min=1,
            max=168
        )
steps.add_class("right-space-class")
display(HTML("<style>.right-space-class{margin-right: 50px;}</style>"))

periods = widgets.BoundedIntText(
            value=10,
            min=1,
            max=100,
            step=1,
            description='Number of periods to aggregate',
            disabled=False,
            style={'description_width': 'initial'}
        )
periods.add_class("right-space-class")
display(HTML("<style>.right-space-class{margin-right: 50px;}</style>"))

algorithm = widgets.Dropdown(
                options=[alg for alg in tsam.TimeSeriesAggregation.CLUSTER_METHODS],
                value='hierarchical',
                description='Algorithm',
                disabled=False,
                style={'description_width': 'initial'}
            )
algorithm.add_class("right-space-class")
display(HTML("<style>.right-space-class{margin-right: 50px;}</style>"))

attribute = widgets.Dropdown(
                options=[('...')],
                value='...',
                description='Attribute',
                disabled=False,
                style={'description_width': 'initial'}
            )
attribute.add_class("right-space-class")
display(HTML("<style>.right-space-class{margin-right: 50px;}</style>"))

button_agr = Button(description="Aggregate", button_style="success", icon='check')

HBox(children=[steps, periods, algorithm, attribute, button_agr])

In [8]:
plot1 = Output()
plot1

Output()

In [9]:
def create_download_link(df, title, filename = "data_s_"+str(steps.value)+"_p_"+str(periods.value)+"_a_"+algorithm.value+".csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a class="btn btn-success" download="{filename}" href="data:text/csv;base64,{payload}" target="_blank" style="display: flex; justify-content: center; align-items: center;">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

In [10]:
def plot_duration(raw,predictedPeriods):
    fig, axes = plt.subplots(figsize = [6, 2], dpi = 100, nrows = 1, ncols = 1)
    raw['Load'].sort_values(ascending=False).reset_index(drop=True).plot(label = 'Original')
    predictedPeriods['Load'].sort_values(ascending=False).reset_index(drop=True).plot(label = '8 typ days')

    plt.legend()
    plt.xlabel('Hours [h]')
    plt.ylabel('Duration Load [MW]')
    
    return fig

In [11]:
def plot_aggregation(predictedPeriods, raw_data):
    fig, axes = plt.subplots(figsize = [6, 2], dpi = 100, nrows = 1, ncols = 1)
    stacked, timeindex = tsam.unstackToPeriods(copy.deepcopy(predictedPeriods[attribute.value]), periods.value)
    cax = axes.imshow(stacked.values.T, interpolation = 'nearest', vmin = raw_data[attribute.value].min(), vmax = raw_data[attribute.value].max())
    axes.set_aspect('auto')  
    axes.set_ylabel('Hour')
    plt.xlabel('Day')

    fig.subplots_adjust(right = 1.2)
    cbar=plt.colorbar(cax)

    if attribute.value == 'T':
        cbar.set_label('T [°C]')
    elif attribute.value == 'GHI':
        cbar.set_label('GHI [W/m²]')
    elif attribute.value == 'Wind':
        cbar.set_label('Wind [m/s]')
    else:
        cbar.set_label('Load [W]')
    
    return fig

In [12]:
@data_output.capture()
def onclick_data(button):
    raw_data = pd.read_csv(StringIO(str(uploader.data[0],'utf-8')), index_col = 0)
               
    row, col = raw_data.shape
    
    periods.max = int(row/steps.value)
    attribute.options = [atr for atr in raw_data.columns]
    
    print("Number of time series: ", col)
    print("Number of time steps: ", row)
    
            
button_data.on_click(onclick_data)

@plot1.capture(clear_output=True)
def onclick_agr(button):
    raw_data = pd.read_csv(StringIO(str(uploader.data[0],'utf-8')), index_col = 0)
    with plot1:
        aggregation = tsam.TimeSeriesAggregation(raw_data, noTypicalPeriods = steps.value, hoursPerPeriod = periods.value, clusterMethod = algorithm.value, solver='gurobi')
        typPeriods = aggregation.createTypicalPeriods()
        predictedPeriods = aggregation.predictOriginalData()        
        
        display(HTML("<h3 style='text-align: center;'>Result plot</h3>"))
        display(plot_aggregation(predictedPeriods, raw_data), plot_duration(raw_data, predictedPeriods))

        display(create_download_link(typPeriods, 'Download TypicalPeriods'))
        display(create_download_link(predictedPeriods, 'Download PredictedPeriods'))
        
button_agr.on_click(onclick_agr)

In [13]:
##### Footer
display(HTML('<hr><a target="_blank" href="http://www.metis-platform.net/metis-platform/DE/_ServicePages/Imprint/_node.html;jsessionid=FFAF99E7AD5DD644EC762EC0096E1C6E">IMPRINT</a>|<a target="_blank" href="http://www.metis-platform.net/metis-platform/DE/_ServicePages/DataProtection/_node.html;jsessionid=FFAF99E7AD5DD644EC762EC0096E1C6E">DATA PROTECTION</a>'))

In [14]:
# !jupyter nbconvert --to script TSAM_Voila-Align.ipynb